# Week 0: Introduction to Deep Learning Frameworks

## Notebook 1: MNIST Classification with a Dense Neural Network on Tensorflow2

Welcome to Applied AI study group! As a starter pack to the study group, you have three notebooks to get acquainted with the commonly used deep learning frameworks.

We will use Python together with Jupyter to go through all our notebooks. You can use the Python3 installed and available on your system, or you can go to the [python website](https://www.python.org/downloads/) to install Python3 to your system. Another alternative is to use [miniconda](https://docs.conda.io/en/latest/miniconda.html) to install python from scratch along with some useful packages.

## 0. Problem Definition

In all of the notebooks of this preparatory week, the problem we are trying to solve is **classification** using machine learning. More specifically, we have images and different categories. We are going to build models that will predict the category of a given image.

The dataset we are using in this notebook is [MNIST](http://yann.lecun.com/exdb/mnist/). This is a widely used classification dataset in computer vision and machine learning fields, consisting of handwritten digits from zero to nine. We will try to train a model that predicts the digit given an image.

## 1. Installation

To install Jupyter notebook on your system, you can run the following command to install with pip:

    pip install notebook
    
Or if you are using conda, you can run:

    conda install -c conda-forge notebook
    
Go to the directory where these notebooks are contained and run:

    jupyter notebook
    
to open up your notebooks and begin your adventure!

## 2. Imports and Checks

In this notebook, we are starting our journey of deep learning frameworks with [TensorFlow2](https://www.tensorflow.org). We first install TensorFlow2 using the official guidelines found [here](https://www.tensorflow.org/install).

The whole process usually boils down to running:

In [ ]:
!pip install tensorflow

After the installation is done, let's import tensorflow first:

In [ ]:
import tensorflow as tf

Let's print the version and make sure that we are using the right version:

In [ ]:
print(tf.__version__)

If you are seeing any version above >= 2.0.0, then you are good to go.

Below we are installing and importing a high level wrapper around `tf.data` named `tensorflow_datasets` to directly load datasets that are ready to be trained! We will only use this package to show the list of datasets available within `tf.data`:

In [ ]:
!pip install tensorflow_datasets

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
tfds.list_builders()

Below we import the necessary libraries for data exploration and some further data operations. If any of these packages are not installed on your system, please install them via `pip` or `conda`:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

## 3. Data Preparation

We will use `tensorflow_datasets` to load the MNIST dataset. MNIST may be the most commonly used dataset in computer vision because of its simplicity. We are splitting the data as *train* and *test* sets and we are not using batching yet, so the `batch_size` parameter is -1.

In [ ]:
mnist_training, mnist_test = tfds.load('mnist', split=['train', 'test'], batch_size=-1, as_supervised=True)                     

Below we see a summary of the pixel values of the MNIST data:

In [ ]:
print(mnist_training)

In [ ]:
print(mnist_test)

We get the images and labels separately to prepare for training:

In [ ]:
mnist_training_images, mnist_training_labels = mnist_training[0], mnist_training[1]
mnist_test_images, mnist_test_labels = mnist_test[0], mnist_test[1]

As the next step, we print the shapes. MNIST contains $28 \times 28$ grayscale images. In addition, we have 60,000 training images and 10,000 test images.

In [ ]:
print(mnist_training_images.shape)
print(mnist_training_labels.shape)

print(mnist_test_images.shape)
print(mnist_test_labels.shape)

Let's visualize the first training image using `matplotlib`:

In [ ]:
plt.imshow(mnist_training_images[0][:, :, 0] ,cmap = 'gray')
print(mnist_training_labels[0])

Let's also visualize the first test image:

In [ ]:
plt.imshow(mnist_test_images[0][:, :, 0] ,cmap = 'gray')
print(mnist_test_labels[0])

Next, we begin data preprocessing. We will use `tf.reshape` to change the shapes of the images into trainable vectors of size 784 (28 x 28). First we get the shapes separately:

In [ ]:
num_training_images = mnist_training_images.shape[0]
num_test_images = mnist_test_images.shape[0]

img_width, img_height = mnist_training_images.shape[1], mnist_training_images.shape[2]

Since we are using a dense network, we *flatten* the images into vectors of $784 \times 1$:

In [ ]:
mnist_training_images = tf.reshape(mnist_training_images, shape=(num_training_images, img_width * img_height))
mnist_test_images = tf.reshape(mnist_test_images, shape=(num_test_images, img_width * img_height))

Let's see if we actually changed the shape of the data:

In [ ]:
print(mnist_training_images.shape)
print(mnist_test_images.shape)

Another preprocessing step is to normalize the data. As you already know from studying deep learning, normalization is a key step on preparing the data. MNIST pixels are normally between 0 and 255. We normalize the images by dividing each pixel to 255 to map the pixel values between 0 and 1.

Let's first look at the minimum and maximum values of the pixels and the labels.

Please note that we do not have to normalize the labels. However, we need to create one-hot vectors from label values. More on that in a short while:

In [ ]:
print(np.amax(mnist_training_images[0]),np.amin(mnist_training_images[0]))
print(np.amax(mnist_test_images[0]),np.amin(mnist_test_images[0]))

print(np.amax(mnist_training_labels),np.amin(mnist_training_labels))
print(np.amax(mnist_test_labels),np.amin(mnist_test_labels))

We divide all the pixel values to 255.0 and cast them to type `tf.float32`. We also cast the label values into `tf.int64`.

In [ ]:
def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)

    return x, y

We have to create one-hot vectors for the labels for the neural network to calculate the error. Below, we are creating the one-hot vectors and actually creating the dataset with the batch size 128:

In [ ]:
def create_dataset(xs, ys, n_classes=10):
    ys = tf.one_hot(ys, depth=n_classes)
    
    return tf.data.Dataset.from_tensor_slices((xs, ys)) \
    .map(preprocess) \
    .shuffle(len(ys)) \
    .batch(128)

In [ ]:
train_dataset = create_dataset(mnist_training_images, mnist_training_labels)
test_dataset = create_dataset(mnist_test_images, mnist_test_labels)

In [ ]:
print(train_dataset)
train_dataset.element_spec    

Yay! We have our dataset now. Let's check the dataloader:

In [ ]:
batch_images, batch_labels = next(iter(train_dataset))
print(batch_images.shape)
print(batch_labels.shape)
print(np.amax(batch_images[0]),np.amin(batch_images[0]))

Our data loader works like a charm. We have 128 vectors that are 784 dimensional as images, and 128 vectors that are 10 dimensional as labels.

Our maximum pixel value is 1 and the minimum is 0. Data is ready!

Let's visualize the first image in our batch:

In [ ]:
plt.imshow(tf.reshape(batch_images[0], shape=(img_width, img_height, 1))[:, :, 0] ,cmap = 'gray')

## 4. Model Creation

Let's define the hyperparameters of the model that we are going to use. We will create a three layer neural network consisting of dense layers. The `layer_neurons` variable below defines the sizes of the network.

In [ ]:
input_shape = 784
label_shape = 10

lr = 0.003

layer_neurons = [
    [input_shape, 200],
    [200, 80],
    [80, label_shape],
]

bias_shapes = [200, 80, label_shape]
initializer = tf.initializers.glorot_uniform()

Below we define a function that creates a dense layer in TF2. It is simply multiplying the inputs and the weights, adds biases and passes the whole calculation from a sigmoid.

In [ ]:
def dense_layer(inputs, weights, bias):
    return tf.nn.sigmoid(tf.matmul(inputs, weights) + bias)

Below we write functions to initialize the weights and biases:

In [ ]:
def get_weight(shape, name):
    return tf.Variable(initializer(shape), name=name, trainable=True, dtype=tf.float32)

def get_bias(shape, name):
    return tf.Variable(initializer([shape]), name=name, trainable=True, dtype=tf.float32)

We define the weights and biases to be used in our model:

In [ ]:
weights = []
bias = []
i = 0
for layer in layer_neurons:
    weights.append(get_weight(layer, 'weight{}'.format(i)))
    i+=1

i = 0
for layer in bias_shapes:
    bias.append(get_bias(layer, 'bias{}'.format(i)))
    i+=1


As an important step, we define the function that creates our whole neural network. As mentioned earlier, we have a three layer neural network:

In [ ]:
def model(input):
    l1 = dense_layer(input, weights[0], bias[0])
    l2 = dense_layer(l1, weights[1], bias[1])
    l3 = dense_layer(l2, weights[2], bias[2])
    return l3

Below we define the optimizer and the loss function.

One thing to note here is that since we are using `softmax_cross_entropy_with_logits` as the loss function, we don't have to include a `softmax` layer into our model. The reason for this is that the `softmax_cross_entropy_with_logits` function already applies a softmax to the given inputs.

In [ ]:
optimizer = tf.optimizers.Adam(lr)

def loss(pred, target):
    return tf.nn.softmax_cross_entropy_with_logits(target, pred)

We define one training step below. Note that we are using `tf.GradientTape` here for automatic differentiation. Therefore, we don't have to define the backward pass operations while creating the model.

In [ ]:
def train_step(model, inputs, outputs, epoch):
    epoch_loss_avg = None
    
    with tf.GradientTape() as tape:
        current_loss = loss(model(inputs), outputs)
        grads = tape.gradient(current_loss, weights)
        optimizer.apply_gradients(zip(grads, weights))
    
    epoch_loss_avg = tf.reduce_mean(current_loss)
    
    return epoch_loss_avg

## 5. Training

Below we train our model for 10 epochs. We traverse over all training dataset.

Total loss is divided by number of iterations to get average loss for each batch:

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = 0
    i = 0
    for train_data in train_dataset:
        batch_images, batch_labels = train_data
        iter_loss = train_step(model, batch_images, batch_labels, epoch)
        epoch_loss += iter_loss
        i+=1
    print("--- On epoch {} ---".format(epoch))
    tf.print("| Loss: ", epoch_loss/i)

## 6. Evaluation

We use the trained model over the test dataset and normalize with number of test samples to obtain the final accuracy:

In [ ]:
acc = 0 
for test_data in test_dataset:
    batch_images, batch_labels = test_data
    predictions = model(batch_images)
    predictions = tf.nn.softmax(predictions)
    equality = tf.math.equal(np.argmax(predictions, axis=1), np.argmax(batch_labels, axis=1))
    acc += np.sum(equality)
acc /= 10000
print(acc)

Congratulations on finishing this notebook! You can move on to the next one, which we are going to use PyTorch to classify MNIST examples.

**Bonus - Try to:**

- Get a test image
- Plot the image
- Make a model prediction on the image
- Print the predicted label and the actual label!